In [1]:
!pip uninstall -y transformers accelerate peft sentence-transformers
!pip install transformers==4.38.2 datasets==2.18.0 accelerate==0.28.0 peft==0.9.0 sentence-transformers==2.2.2

Found existing installation: transformers 4.53.3
Uninstalling transformers-4.53.3:
  Successfully uninstalled transformers-4.53.3
Found existing installation: accelerate 1.9.0
Uninstalling accelerate-1.9.0:
  Successfully uninstalled accelerate-1.9.0
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.

In [2]:
import os
import torch
import shutil
from tqdm.auto import tqdm
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

def fine_tune_flan_t5_optimized():
    # --- 1️⃣ Setup ---
    model_name = "google/flan-t5-base"
    dataset_name = "cnn_dailymail"
    dataset_version = "3.0.0"
    cache_dir = "/kaggle/working/tokenized_cnn"
    output_dir = "/kaggle/working/flan-t5-cnn-dailymail-model"

    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"GPUs detected: {torch.cuda.device_count()}")

    # --- 2️⃣ Load Dataset ---
    print("\n📦 Loading dataset...")
    train_split = load_dataset(dataset_name, dataset_version, split="train[:100000]")
    validation_split = load_dataset(dataset_name, dataset_version, split="validation")
    test_split = load_dataset(dataset_name, dataset_version, split="test")

    dataset = DatasetDict({
        "train": train_split,
        "validation": validation_split,
        "test": test_split
    })

    # --- 3️⃣ Tokenizer & Model ---
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # --- 4️⃣ Preprocessing Function ---
    def preprocess_function(examples):
        prefix = "summarize: "
        inputs = [prefix + doc for doc in examples["article"]]
        model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    # --- 5️⃣ Cache Handling ---
    rebuild_cache = False
    if os.path.exists(cache_dir):
        try:
            print("\n📂 Loading tokenized dataset from cache...")
            tokenized_datasets = load_from_disk(cache_dir)
            # Check if splits are non-empty
            if any(len(split) == 0 for split in tokenized_datasets.values()):
                print("⚠️ Cache appears empty — rebuilding...")
                rebuild_cache = True
        except Exception as e:
            print(f"⚠️ Cache corrupted ({e}) — rebuilding...")
            rebuild_cache = True
    else:
        rebuild_cache = True

    # --- 6️⃣ Tokenization Step (if needed) ---
    if rebuild_cache:
        if os.path.exists(cache_dir):
            shutil.rmtree(cache_dir)
        print("⚙️ Tokenizing dataset — this may take a few minutes...")
        tokenized_datasets = dataset.map(
            preprocess_function,
            batched=True,
            num_proc=4,
            remove_columns=dataset["train"].column_names,
            desc="Tokenizing"
        )
        tokenized_datasets.save_to_disk(cache_dir)
        print("✅ Tokenized dataset saved to cache.")
    else:
        print("✅ Using cached dataset.")

    print({k: len(v) for k, v in tokenized_datasets.items()})

    # --- 7️⃣ Training Arguments ---
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        num_train_epochs=1,
        per_device_train_batch_size=4,   # Larger batch = faster training
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        save_total_limit=2,
        fp16=True,
        bf16=False,
        optim="adafactor",              # ✅ Adafactor saves VRAM
        evaluation_strategy="steps",
        eval_steps=2000,
        save_steps=2000,
        logging_steps=200,
        predict_with_generate=True,
        report_to="none",
        load_best_model_at_end=True
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    # --- 8️⃣ Trainer ---
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # --- 9️⃣ Train ---
    print("\n🚀 Starting fine-tuning...")
    trainer.train()

    # --- 🔟 Save Model ---
    final_model_path = f"{output_dir}/final"
    print("\n💾 Saving model...")
    trainer.save_model(final_model_path)

    # --- 🔁 Zip Model ---
    print("\n📦 Creating ZIP archive...")
    shutil.make_archive("/kaggle/working/my_final_cnn_model", 'zip', final_model_path)
    print("✅ Model archived at /kaggle/working/my_final_cnn_model.zip")

if __name__ == "__main__":
    fine_tune_flan_t5_optimized()


2025-10-14 05:20:20.904033: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760419221.330685      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760419221.451449      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CUDA available: True
GPUs detected: 2

📦 Loading dataset...


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

...


🚀 Starting fine-tuning...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
2000,1.017900,1.056085
4000,0.999600,1.056477
6000,1.004400,1.052414


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].



💾 Saving model...

📦 Creating ZIP archive...
✅ Model archived at /kaggle/working/my_final_cnn_model.zip
